In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

def temporal_pearson_correlation(user1, user2, data, decay_factor=0.9):
    # Calculates Pearson correlation with temporal decay applied to ratings
    ratings1 = data[data['userId'] == user1]
    ratings2 = data[data['userId'] == user2]
    common_movies = pd.merge(ratings1, ratings2, on='movieId', how='inner')

    if len(common_movies) == 0:
        return 0

    latest_timestamp = max(common_movies['timestamp_x'].max(), common_movies['timestamp_y'].max())
    common_movies['decay'] = decay_factor ** (latest_timestamp - common_movies[['timestamp_x', 'timestamp_y']].max(axis=1))
    weighted_diff = (common_movies['rating_x'] - common_movies['rating_y']) * common_movies['decay']
    sim = np.dot(weighted_diff, weighted_diff) / (np.linalg.norm(weighted_diff) ** 2)

    return 1 / (1 + sim)

def get_similar_users(target_user, data, similarity_func, top_n=10):
    # Finds similar users based on the specified similarity function
    if target_user not in data['userId'].unique():
        raise ValueError(f"User {target_user} not found in the dataset.")
    user_ids = data['userId'].unique()
    similarities = [(user, similarity_func(target_user, user, data)) for user in user_ids if user != target_user]
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n][0]

def get_user_recommendations(user, data, similarity_func, top_n=10):
    # Gets recommendations for a user
    similar_users = get_similar_users(user, data, similarity_func, top_n)
    user_movie_ratings = data[data['userId'] == user]
    recommendations = {}
    for similar_user in similar_users:
        similar_user_ratings = data[data['userId'] == similar_user]
        for _, row in similar_user_ratings.iterrows():
            movie_id = row['movieId']
            if movie_id not in user_movie_ratings['movieId'].values:
                recommendations.setdefault(movie_id, []).append(row['rating'])
    return dict(sorted(recommendations.items(), key=lambda x: np.mean(x[1]), reverse=True)[:top_n])

def get_group_recommendations(group, data, similarity_func, top_n=10):
    # Aggregates recommendations for a group
    all_recommendations = {}
    for user in group:
        user_recs = get_user_recommendations(user, data, similarity_func, top_n)
        for movie, rating in user_recs.items():
            all_recommendations.setdefault(movie, []).extend(rating)
    return dict(sorted(all_recommendations.items(), key=lambda x: np.mean(x[1]), reverse=True)[:top_n])

# Load the dataset
data = pd.read_csv('ratings.csv')

# Define the group of users
group = [6, 5, 4]

# Displaying the top-10 temporal group recommendations for the group in 3 different sequences
sequences = []
decay_factors = [0.9, 0.6, 0.4]  # Different decay factors for different sequences

for decay in decay_factors:
    recommendations = get_group_recommendations(group, data, lambda u1, u2, d: temporal_pearson_correlation(u1, u2, d, decay_factor=decay), top_n=10)
    sequences.append(recommendations)

for i, sequence in enumerate(sequences, start=1):
    print(f"Sequence {i} Recommendations:")
    for movie_id, score in sequence.items():
        print(f"Movie ID: {movie_id}, Score: {score}")
    print("\n")



Sequence 1 Recommendations:
Movie ID: 25.0, Score: [5.0]
Movie ID: 62.0, Score: [5.0]
Movie ID: 175.0, Score: [5.0]
Movie ID: 288.0, Score: [5.0]
Movie ID: 356.0, Score: [5.0]
Movie ID: 553.0, Score: [5.0]
Movie ID: 50.0, Score: [5.0]
Movie ID: 101.0, Score: [5.0]
Movie ID: 151.0, Score: [5.0]
Movie ID: 157.0, Score: [5.0]


Sequence 2 Recommendations:
Movie ID: 101.0, Score: [5.0, 5.0]
Movie ID: 157.0, Score: [5.0, 5.0]
Movie ID: 260.0, Score: [5.0, 5.0]
Movie ID: 661.0, Score: [5.0]
Movie ID: 919.0, Score: [5.0]
Movie ID: 923.0, Score: [5.0]
Movie ID: 940.0, Score: [5.0]
Movie ID: 954.0, Score: [5.0]
Movie ID: 1023.0, Score: [5.0]
Movie ID: 1024.0, Score: [5.0]


Sequence 3 Recommendations:
Movie ID: 47.0, Score: [5.0]
Movie ID: 101.0, Score: [5.0, 5.0]
Movie ID: 151.0, Score: [5.0, 5.0]
Movie ID: 157.0, Score: [5.0, 5.0]
Movie ID: 163.0, Score: [5.0, 5.0]
Movie ID: 216.0, Score: [5.0, 5.0]
Movie ID: 231.0, Score: [5.0, 5.0]
Movie ID: 260.0, Score: [5.0]
Movie ID: 333.0, Score: [5.0,